In [1]:
import openml
import pandas as pd
import numpy as np
import json

In [2]:
openml.config.apikey = '7b79300852166039a83f129b8898f9f5'
np.random.seed(1)

In [3]:
dataset = openml.datasets.get_dataset(1590)
(X, y, categorical, names) = dataset.get_data(
    target=dataset.default_target_attribute,
    return_categorical_indicator=True,
    return_attribute_names=True,
)

In [4]:
vals = {}
for i, name in enumerate(names):
    vals[name] = X[:, i]
vals["target"] = y
df = pd.DataFrame(vals)

# Metadane

In [44]:
final_json = {
    "id": "openml_adult",
    "added_by": "Siemashko",
    "date": "20-02-2019",
    "name": "adult",
    "source": "openml",
    "url": "https://www.openml.org/d/1590",
    "number_of_features": len(names),
    "number_of_instances": df.shape[0],
    "number_of_missing_values": int(df.isna().sum().sum()),
    "number_of_instances_with_missing_values": df[df.isna().any(axis=1)].shape[0],
    "variables": {}
}

In [45]:
def parse_np_matrix_to_json(a):
    output = {}
    for i in range(len(a[0])):
        output[str(a[0][i])] = int(a[1][i])
    return output

def summarize_categorical_variable(values, name, final_json):
    final_json['variables'][name] = {
        "name": name,
        "type": "categorical",
        "number_of_unique_values": len(np.unique(values.dropna())),
        "number_of_missing_values": int(values.isna().sum()),
        "cat_frequencies": parse_np_matrix_to_json(np.unique(values.dropna(), return_counts=True)),
        "num_minimum": None,
        "num_1qu": None,
        "num_median": None,
        "num_mean": None,
        "num_3qu": None,
        "num_maximum": None
    }
    
def summarize_numerical_variable(values, name, final_json):
    final_json['variables'][name] = {
        "name": name,
        "type": "numerical",
        "number_of_unique_values": len(np.unique(values.dropna())),
        "number_of_missing_values": int(values.isna().sum()),
        "cat_frequencies": None,
        "num_minimum": float(np.min(values.dropna())),
        "num_1qu": float(np.percentile(values.dropna(),25)),
        "num_median": float(np.percentile(values.dropna(),50)),
        "num_3q": float(np.percentile(values.dropna(),75)),
        "num_maximum": float(np.max(values.dropna()))
    }

# Podsumowanie zmiennych

In [47]:
for i in range(len(names)):
    if categorical[i]:
        summarize_categorical_variable(df.loc[:,names[i]], names[i], final_json)
    else:
        summarize_numerical_variable(df.loc[:,names[i]], names[i], final_json)

# zapis do pliku

In [51]:
with open('dataset.json', 'w') as fp:
    json.dump([final_json], fp, indent=4)